In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', None)

C:\Users\seppa\AppData\Local\Temp\ipykernel_224024\4261717029.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


<h3>Funktiot

In [3]:
#Lyhentää minkä tahansa merkkijonon oletuksena 20 merkkiin
def shorten(value, length = 20):
    if isinstance(value, str) and len(value) > length:
        return value[:length]
    return value

In [4]:
#Splittaa tekstin ison kirjaimen kohdalta. Palauttaa 2: (tehty vain yhtä tarkoitusta varten)
def split_by_upper(text):
    parts = re.split(r'(?=[A-Z])', text)
    return parts[2:]

In [5]:
#Jakaa merkkijonon pilkun kohdalta, palauttaa ekan osion
def split_by_comma(text):
    if text != None:
        parts = text.split(',')
    else:
        parts = ['Tuntematon']
    return parts[0]

In [6]:
#Jakaa merkkijonon annetun merkin kohdalta, palauttaa halutun kohdan
def splitter(text, by_what, return_what = 0):
    if text != None:
        parts = text.split(by_what)   
    else:
        parts = []
    return parts[return_what]    

In [7]:
#Jakaa merkkijonon kauttaviivan kohdalta, palauttaa ekan kohdan
def split_by_dash(text):
    if text != None:
        parts = text.split('/')
    else:
        parts = ['Tuntematon']
    return parts[0]

In [8]:
#Muokkaa väliviivan jälkeisen ison kirjaimen pieneksi. (Tehty kaupunginosaa varten, ei ehkä käytössä enää??)
def replace_upper_after_hyphen(text):
    return re.sub(r'-(\w)', lambda m: '-' + m.group(1).lower(), text)

In [9]:
#Etsii merkkijonosta samanlaisen listasta. Muuttaa merkkijonoon listan tekstin.
def find_and_replace_material(description, list_of_types):
    if isinstance(description, str):
        for item in list_of_types:
            if item in description.lower():
                return item
    return None

In [10]:
#Näillä matchataan postinumeron perusteella oikea kaupunginosa
locations = {
    90100: "Keskusta",90120: "Heinäpää",90130: "Intiö",90140: "Karjasilta",90150: "Höyhtyä",90220: "Kontinkangas",
    90230: "Oulunsuu",90240: "Maikkula",90250: "Kaukovainio",90310: "Madekoski",90400: "Nuottasaari",90410: "Kiviniemi",
    90420: "Kaakkuri",90460: "Oulunsalo",90470: "Varjakka",90500: "Tuira",90510: "Hietasaari",90520: "Toppila",
    90530: "Alppila",90540: "Kuivasjärvi",90550: "Puolivälinkangas",90560: "Koskela",90570: "Kaijonharju",90580: "Rajakylä",
    90590: "Teknologiakylä",90600: "Postikeskus",90620: "Rusko",90630: "Korvensuora",90650: "Myllyoja",90660: "Sanginsuu",
    90670: "Hiukkavaara",90800: "Pateniemi",90810: "Kiviniemi",90820: "Kello",90830: "Haukipudas",90840: "Haukipudas",
    90850: "Martinniemi",90860: "Halosenniemi",90900: "Kiiminki",90910: "Kontio",90940: "Jääli",91200: "Yli-Ii",91240: "Tannila",91260: "Pahkakoski",
    91300: "Ylikiiminki",91310: "Arkala"
}

<h3>Ohjelma

<h4>Uuden ja vanhan df:n yhdistäminen

<li>Lue uusi ja vanha csv df:ksi
<li>Lisää vanhaan df:ään kaikki kohteet, mitkä eivät ole siellä
<li>Merkkaa myydyksi kaikki vanhan df:n kohteet, jotka eivät ole uudessa


In [11]:
# Step 1: Read existing data and new weekly data into DataFrames
vanha_csv = 'combined_listings.csv'
uusi_csv = '24082024_data.csv'

df_old = pd.read_csv(vanha_csv)
df_new = pd.read_csv(uusi_csv)


In [12]:
#muutetaan nimet pieniksi kirjaimiksi ja välit alaviivoiksi
df_old = df_old.rename(columns=lambda x: x.lower())
df_old.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
df_new = df_new.rename(columns=lambda x: x.lower())
df_new.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [13]:
# Ensure 'kohdenumero' and 'haettu_pvm' columns exist in both DataFrames
if 'kohdenumero' not in df_old.columns or 'kohdenumero' not in df_new.columns:
    raise ValueError("Column 'kohdenumero' must be present in both dataframes")

if 'haettu_pvm' not in df_new.columns:
    raise ValueError("Column 'haettu_pvm' must be present in the new dataframe")

In [14]:
# Step 2: Identify and merge only new listings
new_listings = df_new[~df_new['kohdenumero'].isin(df_old['kohdenumero'])]

In [15]:
new_listings

,kohdenumero,sijainti,tyyppi,omistusmuoto,huoneistoselitelmä,huoneita,asuintilojen_pinta-ala,kerrokset,asuinkerrosten_määrä,rakennusvuosi,varaustilanne,valinnainen_vuokratontti,vastike,muut_maksut,sauna,parveke,tietoliikenne,asunnon_kunto,lämmitysjärjestelmän_kuvaus,rakennus-_ja_pintamateriaalit,keittiön_kuvaus,kylpyhuoneen_kuvaus,wc-tilojen_kuvaus,saunan_kuvaus,olohuoneen_kuvaus,makuuhuoneiden_kuvaus,säilytystilojen_kuvaus,taloyhtiön_nimi,tontin_omistus,tontin_vuokra,tontin_vuokra-aika_päättyy,lisätietoa_auton_säilytyksestä,hinta,url,haettu_pvm,myyty_pvm,myyty,y_tunnus,kattotyyppi,isännöitsijän_yhteystiedot,muuta_taloyhtiöstä,tehdyt_remontit,tulevat_remontit,tontin_koko,kohteen_lisätiedot,hissi,taloyhtiön_autopaikat,muiden_tilojen_pinta-ala,lisätietoja_kunnosta,tulisija,vesijohto,viemäri,sähköliittymä_siirtyy,siirtyvät_liittymät,ilmanvaihto,kodinhoitohuoneen_kuvaus,tontin_tyyppi,tontin_pinta-ala,rakennukset,palvelut,liikenneyhteydet,näkymät,pihan_kuvaus,muuta_kauppaan_kuuluvaa,lisätietoa_tontista,tontin_vuokra-aika,lisätietoja,lämmitysjärjestelmä,rakennusvaihe,kattotyypin_kuvaus,tilojen_kuvaus,vesihuollon_kuvaus,kiinteistötunnus,lisätietoja_tontista,rakennusoikeuden_pinta-ala,tehokkuusluku,lisätietoja_rakennusoikeudesta,tiedustelut,lisätietoa_tontin_omistuksesta,kosteustarkastus,ranta,asuntoon_kuuluu,rakennuttaja,kuntotarkastus,tie_perille,asunnon_käytössä_olevat_autopaikat,rasitteet,vuokrasopimus_päättyy,ajo-ohjeet,asbestikartoitus,lisätietoja_vuokra-ajasta,tontin_nimi,käyttö-_ja_luovutusrajoitukset,rasitukset_ja_rasitteet
4,21301565,"OuluHöyhtyäHanhitie 13, 90150 Oulu",Kerrostalo(huoneisto),Omistusasunto,"2h, kk. s",Kaksio,"46,5 m²",3/4,NaN,2016,NaN,NaN,"Hoitovastike190,65 € / kkPääomavastike767,52 €...","Vesimaksu18,00 € / kkLisätietoa maksuistaVesim...",Asunnossa on sauna,Asunnossa on parveke,"kaapeli-tv, kaapeli-tv taloyhtiössä",Hyvä,KaukolämpöAsunnon tilat ja materiaalit,Pääasiallinen rakennusmateriaali:Betoni,Lattiamateriaalit: parkettiSeinämateriaalit: m...,Seinämateriaali: kaakeliLattiamateriaali: laat...,NaN,Lattiamateriaalit: laattaSeinämateriaalit: puu...,Lattiamateriaalit: parkettiSeinämateriaalit: m...,Lattiamateriaalit: parkettiSeinämateriaalit: m...,NaN,Asunto Oy Höyhtyän Pihlaja,Vuokra,NaN,2.12.2075Luotettava nettiyhteys kotiin – tutus...,NaN,168000,https://www.etuovi.com/kohde/21301565,24082024,NaN,0,NaN,Aumakatto,REIM Oulu Oy Kari Sutelainen kari.sutelainen@r...,Muuta: Hallinnanjakosopimus baapuritalojen kan...,NaN,"Kunnossapitotarpeet, jotka on esitetty yhtiöko...",6 146 m²,NaN,Taloyhtiössä on hissi,Autopistokepaikkoja: 6\nAutohallipaikkoja: 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,68480305,"OuluKirkkokangasLatvustie 11, 90650",Omakotitalo(kiinteistö),Omistusasunto,4-5h + k + khh / ph + sa + 2wc + autotalli + a...,5 huonetta,106 m²,NaN,NaN,1982,NaN,NaN,NaN,"Kiinteistövero231,00 € / vuosiMuut lämmityskul...",Asunnossa on sauna,NaN,NaN,Hyvä,Lämmitystapa: KaukolämpöLämmönjakotapa: Vesiki...,"Pääasiallinen rakennusmateriaali:Puu, Puurunko...",Seinämateriaali: Maalattu seinäLattiamateriaal...,Seinämateriaali: KaakeliseinäLattiamateriaali:...,Erillinen kalustettu wcToinen wc-tila pesuhuon...,Seinämateriaali: PaneeliseinäLattiamateriaali:...,Seinämateriaali: Maalattu seinäLattiamateriaal...,Lattiamateriaali: ParkettilattiaMukavan kokois...,NaN,NaN,Vuokra,488 € / vuosiPalvelut ja liikenneyhteydet,NaN,NaN,209000,https://www.etuovi.com/kohde/68480305,24082024,NaN,0,NaN,Harjakatto,NaN,NaN,2023 pesuhuone uusittu. 2021 saunaremontti. 20...,NaN,NaN,Asunnossa on terassi.Terassin ilmansuunta: Län...,NaN,NaN,42 m²,Hyvin pidetty ja hoidettu koti,TakkaVaraava takka,Kunnan vesijohto,Kunnan,Kyllä,"Kaukolämpösopimus, Sähköliittymä, Vesiliittymä...","LämmöntalteenottoKoneellinen ilmanvaihto, LTOA...",Toimiva tila pesuhuoneessa. Paikka pyykkitorni...,Tasamaatontti,737 m²,"Autotall

In [16]:
#muodostetaan uusista kohteista oma csv, jonka perusteella ladataan aina kuvat eri ohjelmassa
new_listings.to_csv('hae_kuvat.csv', index=False)

In [17]:
df_combined = pd.concat([df_old, new_listings], ignore_index=True)

In [18]:
#Temp to replace nan with y_tunnus
#y_tunnus_map = df_new.set_index('kohdenumero')['y_tunnus'].to_dict()
#df_combined['y_tunnus'] = df_combined['kohdenumero'].map(y_tunnus_map).combine_first(df_combined['y_tunnus'])


In [19]:
df_combined['y_tunnus'].isna().sum()

973

In [20]:
# Step 3: Identify listings not in the new file and mark them
not_in_new = df_old[~df_old['kohdenumero'].isin(df_new['kohdenumero'])]
haettu_pvm = df_new['haettu_pvm'].iloc[0]  # Assuming all rows in the new file have the same 'haettu_pvm' date

df_combined['myyty_pvm'] = df_combined.apply(
    lambda row: haettu_pvm if row['kohdenumero'] in not_in_new['kohdenumero'].values else row.get('myyty_pvm', None),
    axis=1
)
df_combined['myyty'] = df_combined.apply(
    lambda row: 1 if row['kohdenumero'] in not_in_new['kohdenumero'].values else row.get('myyty', 0),
    axis=1
)

In [21]:
# Step 4: Save the combined DataFrame back to a CSV file
df_combined.to_csv('combined_listings.csv', index=False)

print("Combined data has been saved to 'combined_listings.csv'")

Combined data has been saved to 'combined_listings.csv'


<h3>Data Cleaning
   

In [99]:
df = pd.read_csv("combined_listings.csv")

In [100]:
df['y_tunnus'].isna().sum()

973

In [101]:
df['myyty_pvm'].value_counts()

myyty_pvm
24082024.00    377
17082024.00      6
12082024.00      5
5082024.00       2
2082024.00       1
Name: count, dtype: int64

In [102]:
#Karsitaan turhia sarakkeita, joista ei saa mitään hyödyllistä irti
df.columns

Index(['kohdenumero', 'sijainti', 'tyyppi', 'omistusmuoto',
       'huoneistoselitelmä', 'huoneita', 'asuintilojen_pinta-ala', 'kerrokset',
       'rakennusvuosi', 'rakennusvaihe', 'varaustilanne',
       'valinnainen_vuokratontti', 'vastike', 'muut_maksut', 'sauna',
       'parveke', 'hissi', 'asunnon_kunto', 'lämmitysjärjestelmän_kuvaus',
       'rakennus-_ja_pintamateriaalit', 'keittiön_kuvaus',
       'kylpyhuoneen_kuvaus', 'saunan_kuvaus', 'olohuoneen_kuvaus',
       'makuuhuoneiden_kuvaus', 'kattotyyppi', 'kattotyypin_kuvaus',
       'taloyhtiön_nimi', 'muuta_taloyhtiöstä', 'tontin_koko',
       'tontin_omistus', 'lisätietoa_tontin_omistuksesta', 'tontin_vuokra',
       'tontin_vuokra-aika_päättyy', 'palvelut', 'liikenneyhteydet',
       'lisätietoa_auton_säilytyksestä', 'hinta', 'url', 'haettu_pvm',
       'myyty_pvm', 'myyty', 'kohteen_lisätiedot', 'lisätietoja_kunnosta',
       'säilytystilojen_kuvaus', 'isännöitsijän_yhteystiedot',
       'tehdyt_remontit', 'tulevat_remontit'

In [103]:
#Poistetaan kasa turhia
df = df.drop(columns = ['omistusmuoto','muut_maksut','rakennus-_ja_pintamateriaalit','keittiön_kuvaus',
       'kylpyhuoneen_kuvaus', 'säilytystilojen_kuvaus', 'kattotyyppi','taloyhtiön_autopaikat', 'tehdyt_remontit','palvelut',
       'lisätietoa_auton_säilytyksestä', 'lisätietoja','tietoliikenne', 'muuta_kauppaan_kuuluvaa','kohteen_lisätiedot', 'olohuoneen_kuvaus', 'makuuhuoneiden_kuvaus',
       'muuta_taloyhtiöstä', 'tulevat_remontit', 'tontin_vuokra','tontin_vuokra-aika_päättyy', 'liikenneyhteydet', 'tiedustelut','saunan_kuvaus', 'isännöitsijän_yhteystiedot', 'lisätietoa_tontista',
       'asuntoon_kuuluu', 'tontin_vuokra-aika', 'valinnainen_vuokratontti',
       'lisätietoa_tontin_omistuksesta', 'lisätietoja_kunnosta','wc-tilojen_kuvaus', 'pihan_kuvaus', 'asuinkerrosten_määrä',
       'rakennuttaja', 'tilojen_kuvaus', 'asunnon_käytössä_olevat_autopaikat','näkymät', 'rakennusvaihe', 'varaustilanne', 'kodinhoitohuoneen_kuvaus',
       'kattotyypin_kuvaus', 'kiinteistötunnus', 'asbestikartoitus', 'ranta',
       'muiden_tilojen_pinta-ala', 'lämmitysjärjestelmä', 'lämmitysjärjestelmän_kuvaus','kosteustarkastus','vesihuollon_kuvaus', 'ilmanvaihto', 'vesijohto', 'viemäri',
       'tontin_pinta-ala', 'tie_perille', 'rakennusoikeuden_pinta-ala','lisätietoja_rakennusoikeudesta', 'kuntotarkastus', 'tulisija',
       'ajo-ohjeet'])

In [104]:
df = df.rename(columns={'date': 'haettu_pvm','asuintilojen_pinta-ala': 'koko', 'asunnon_kunto': 'kunto', 'taloyhtiön_nimi': 'taloyhtio'})

In [105]:
df.columns

Index(['kohdenumero', 'sijainti', 'tyyppi', 'huoneistoselitelmä', 'huoneita',
       'koko', 'kerrokset', 'rakennusvuosi', 'vastike', 'sauna', 'parveke',
       'hissi', 'kunto', 'taloyhtio', 'tontin_koko', 'tontin_omistus', 'hinta',
       'url', 'haettu_pvm', 'myyty_pvm', 'myyty', 'siirtyvät_liittymät',
       'tontin_tyyppi', 'tontin_nimi', 'tehokkuusluku', 'rakennukset',
       'vuokrasopimus_päättyy', 'sähköliittymä_siirtyy',
       'käyttö-_ja_luovutusrajoitukset', 'lisätietoja_tontista',
       'lisätietoja_vuokra-ajasta', 'rasitukset_ja_rasitteet', 'rasitteet',
       'y_tunnus'],
      dtype='object')

In [106]:
#Siivotaan rakennusvuosi, tiputetaan vuosiluvuttomat pois
df['rakennusvuosi'] = df['rakennusvuosi'].apply(lambda x: str(x)[:4])
df['rakennusvuosi'] = df['rakennusvuosi'].apply(lambda x: str(x).replace('nan','0')).astype(int)
df = df[df['rakennusvuosi'] > 1900]

### Tyyppi

In [107]:
df['type_temp'] = df['tyyppi'].apply(lambda x: splitter(x, '(',1))
df['type_temp'] = df['type_temp'].apply(lambda x: splitter(x, ')',0))
df['tyyppi'] = df['tyyppi'].apply(lambda x: splitter(x, '(',0))

In [108]:
#Muutetaan tyyppi binäärimuuttujaksi
df['is_apartment'] = df['type_temp'].apply(lambda x: 1 if (x == 'huoneisto') else 0)
df['is_apartment'] = df['is_apartment'].astype(int)
#Poistetaan väliaikainen sarake
df = df.drop(columns =['type_temp'])

<h3>Sijainti

In [109]:
#pilkotaan sijainti postinumeroksi, katuosoitteeksi ja kaupunginosaksi
df['sijainti']

0                   OuluKastelliHoitajanrinne 7 40, 90220
1              OuluKeskustaLinnankatu 23-25 A, 90100 OULU
2                   OuluKeskustaKanavaranta 1, 90100 Oulu
3                   OuluToppilansalmiSiilotie 1 D8, 90520
4                   OuluKeskustaKansankatu 48, 90100 Oulu
5                OuluEtu-LyöttyLyötynkatu 3 A, 90100 OULU
6                         OuluVärttöKurkelankuja 8, 90230
7                OuluHiironenMetsänkuninkaantie 47, 90250
8                  OuluHeinäpääAleksanterinkatu 56, 90120
9                OuluKeskustaRautatienkatu 18, 90100 Oulu
10                OuluKeskustaRautatienkatu 1 A120, 90100
11                       OuluTuiraKoskitie 35, 90500 Oulu
12               OuluToppilansalmiSiilotie 27, 90520 OULU
13                    OuluRajakyläTervakukkatie 46, 90580
14                           OuluIntiöKenttätie 16, 90130
15              OuluToppilansalmiHöyrymyllyntie 14, 90520
16                 OuluKarjasiltaHaasiokatu 8, 90140 Oulu
17            

In [110]:
#muutetaan ensin väliviivan jälkeinen kirjan pieneksi
df['sijainti'] = df['sijainti'].apply(replace_upper_after_hyphen)
#Splitataan sen jälkeen isojen kirjainten kohdalta
df['sijainti_split'] = df['sijainti'].apply(split_by_upper)
#df['sijainti_pituus'] = df['sijainti_split'].apply(lambda x: len(x))

In [111]:
pd.set_option('display.max_rows', 500)
df['kaupunginosa'] = df['sijainti_split'].str.get(0).str.strip('/( ,')

In [112]:
df['kaupunginosa'] = df['kaupunginosa'].apply(lambda x: splitter(x, '/',0))

In [113]:
#Muutetaan NaN ne jotka sisätävät numeroita, eli ovat katuosoitteita
df.loc[df['kaupunginosa'].str.contains(r'\d'),'kaupunginosa'] = np.nan

In [114]:
df['katuosoite'] = df['sijainti_split'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
df['katuosoite'] = df['katuosoite'].apply(split_by_comma)

In [115]:
df['katuosoite']

0           Hoitajanrinne 7 40
1            Linnankatu 23-25 
2                Kanavaranta 1
3                  Siilotie 1 
4                Kansankatu 48
                 ...          
2824            Vesipadantie 2
2825               Kuovitie 8 
2826           Pellikanpolku 3
2827    Tarkka-ampujankatu 20 
2828           Välitalontie 42
Name: katuosoite, Length: 2768, dtype: object

In [116]:
#napataan viiden numeron sarja postinumeroksi
df['postinumero'] = df['sijainti'].str.extract(r'(\b\d{5}\b)')
df['postinumero'] = df['postinumero'].fillna(0)
df['postinumero'] = df['postinumero'].astype(int)

In [117]:
#Muutetaan arvo 'Oulu' -> NaN
df.loc[df['kaupunginosa'] == 'Oulu'] = np.nan

In [118]:
df['kaupunginosa'].isna().sum()

18

In [119]:
#Täytetyään NaN arvot postinumeron perusteella 
mapped_values = df['postinumero'].map(locations)
df['kaupunginosa'] = df['kaupunginosa'].fillna(mapped_values)

In [120]:
#pudotetaan tyhjät pois
df = df.dropna(subset=['kaupunginosa'])

In [121]:
#Tiputetaan tarpeeton pois
df = df.drop('sijainti', axis=1)

<h3>Huoneet

In [122]:
#Otetaan ensimmäinen merkki huoneiden määräksi
df['huoneita'] = df['huoneistoselitelmä'].apply(lambda x: str(x)[0])

In [123]:
#muutetaan puuttuvat nollaksi, ja datatyypin kokonaisluvuiksi
df['huoneita'] = df['huoneita'].apply(lambda x: x if x.isdigit() else '0')
df['huoneita'] = df['huoneita'].astype(int)


In [124]:
#Tiputetaan nollat pois
df = df[df['huoneita'] > 0]

In [125]:
df['koko'] = df['koko'].str.split( ).str.get(0).str.replace(',','.').astype(float)

In [126]:
df['neliohinta'] = df['hinta'] / df['koko']

In [127]:
split_df = df['vastike'].str.split('/', expand=True)

# Assign the split parts to new columns
df['column0'] = split_df[0]


In [128]:
#Erotetaan numerot rivistä
df['hoitovastike'] = df['column0'].str.replace(r'[^0-9,]', '', regex=True).str.replace(',','.')
df['hoitovastike'] = pd.to_numeric(df['hoitovastike'], errors='coerce')
df['hoitovastike'] = np.where(df['hoitovastike'] > 1000, None, df['hoitovastike'].astype(float))
df['hoitovastike'] = pd.to_numeric(df['hoitovastike'], errors='coerce')

In [129]:
#Jos talo ei ole huoneisto, laitetaan vastike nollaksi
df.loc[(df['is_apartment'] == 0) & (df['hoitovastike'].isna()), 'hoitovastike'] = 0

In [130]:
#Tiputetaan rivit joiss tyhjä hoitovastike
df = df.dropna(subset=['hoitovastike'])

In [131]:
#poistetaan turha vastike-kenttä ja aiemmat väliaikaiset
df = df.drop(columns = ['column0','vastike','sijainti_split'])

<h3>Muita muokkauksia

In [132]:
#Kaikki kohdenumerot eivät ole kokonaislukuja
df['kohdenumero'] = df['kohdenumero'].astype(str)

In [133]:
#Muutetaan tieto saunasta ja parvekkeesta 0 tai 1
df['sauna'] = df['sauna'].apply(lambda x: 1 if 'asunnossa on sauna'in str(x).lower() else 0)

In [134]:
df['parveke'] = df['parveke'].apply(lambda x: 1 if 'asunnossa on parveke'in str(x).lower() else 0)

In [135]:
df['tontin_koko'] = df['tontin_koko'].apply(lambda x: str(x).replace("\xa0",''))
df['tontin_koko'] = df['tontin_koko'].str.replace(',','.')
df['tontin_koko'] = df['tontin_koko'].apply(lambda x: str(x).split(' '))
df['tontin_koko'] = df['tontin_koko'].apply(lambda x: x[0]).astype(float)

In [136]:
#Tontin omistus
omistus = ['oma','vuokra']
df['tontin_omistus'] = df['tontin_omistus'].apply(lambda x: find_and_replace_material(x, omistus))

In [137]:
#Luodaan binäärimuuttuja tontin omistuksesta
df['oma_tontti'] = np.where(df['tontin_omistus'] =='oma',1,0)

In [138]:
#Laitetaan vuokratontiksi, jos ei tietoa
df['tontin_omistus'] = df['tontin_omistus'].fillna('vuokra')

In [139]:
#Napataan uudiskohde omaksi binäärimuuttujaksi
df['uudiskohde'] = df['rakennusvuosi'].apply(lambda x: '(Uudiskohde)' in str(x))
df['uudiskohde'] = df['uudiskohde'].astype(int)

<h3>Kerrokset

In [140]:
#Täytetyään tyhjät
df['kerrokset'] = df['kerrokset'].fillna('1/1')

In [141]:
#splitataan tyhjästä merkistä, palautetaan ensimmäinen
df['kerrokset'] = df['kerrokset'].apply(lambda x: splitter(x, ' ',0))

In [142]:
#Korvataan merkkijonoja
df['kerrokset'] = df['kerrokset'].apply(lambda x: x.replace('Ei','1/1'))
df['kerrokset'] = df['kerrokset'].apply(lambda x: x.replace('nan','1/1'))
df['kerrokset'] = df['kerrokset'].apply(lambda x: x.replace('Alin','1/1'))
df['kerrokset'] = df['kerrokset'].apply(lambda x: x.replace('Ylin','1/1'))

In [143]:
#splitataan kauttaviivan kohdalta ja palautetaan kerros
df['kerros'] = df['kerrokset'].apply(lambda x: splitter(x, '/', 0))


In [144]:
#splitataan kauttaviivan kohdalta ja palautetaan kerrosten määrä
df['kerrokset'] = df['kerrokset'].apply(lambda x: splitter(x, '/', 1))
df['kerrokset'] = df['kerrokset'].astype(int)

In [145]:
#Tehdään binäärimuuttuja jos alin kerros
df['kerros'] = df['kerros'].astype(int)
df['alin_kerros'] = df['kerros'].apply(lambda x: 1 if (x == 1) else 0)

In [146]:
#Tehdään binäärimuuttuja jos ylin kerros
df['ylin_kerros'] = 0
df.loc[(df['kerros'] == df['kerrokset']) & (df['kerros'] > 1), 'ylin_kerros'] = 1

<h3>Taloyhtiön nimet

In [147]:
df['taloyhtio'].isna().sum()

442

In [148]:
#muokataan taloyhtiön nimiä samanlaisiksi
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('ASUNTO OY','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('AS OY','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto OY','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto-oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto- osakeyhtiö','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('AsOy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('AS Oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('As Oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('As. Oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('As oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('As ','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('as oy','Asunto Oy'))


df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto-osakeyhtiö','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asuntio Oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto-Osakeyhtiö','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Asunto-Oy','Asunto Oy'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Oulun ',''))

 
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Kiinteistöosakeyhtiö','Kiinteistö Oy'))
#Spesifejä muutoksia
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Pikku-Iikka','Pikku-iikka'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('ISO-IIKKA','Iso-iikka'))
df['taloyhtio'] = df['taloyhtio'].apply(lambda x: str(x).replace('Iso-Iikka','Iso-iikka'))




In [149]:
taloyhtio_filter = df[(df['taloyhtio']== 'nan')&(df['tyyppi'] != 'Omakotitalo')]

In [150]:
#Poistetaan, Oulu joistain taloyhtiöistä
df['taloyhtio'] = df['taloyhtio'].apply(split_by_comma, 0)

C:\Users\seppa\AppData\Local\Temp\ipykernel_224024\1632866936.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['taloyhtio'] = df['taloyhtio'].apply(split_by_comma, 0)


In [151]:
#Tehdään hissistä binäärimuuttuja
df['hissi'] = np.where(df['hissi'] == 'Taloyhtiössä on hissi', 1, 0)

In [152]:
#muutetaan kunto-sarake 
kunnot = ['hyvä','tyydyttävä','välttävä']
#df['makuuhuoneiden_kuvaus'] = df['makuuhuoneiden_kuvaus'].fillna('')
df['kunto'] = df['kunto'].apply(lambda x: find_and_replace_material(x, kunnot))
df['kunto'] = df['kunto'].fillna('hyvä') #Jos ei ole kuntoa niin oletetaan hyväksi
#df['makuuhuoneiden_kuvaus'] = df['makuuhuoneiden_kuvaus'].apply(lambda x: find_and_replace_material(x, lattiat))


In [153]:
#Stripataan tyhjät merkit pois merkkijonojena alusta ja lopusta
df = df.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)

In [154]:
#luodaan csv machine learningia varten
#df.to_csv('ml.csv', index=False)

<h3>Feature engineering

In [155]:
#lasketaan ikä
year = date.today()
year = year.year
df['ika'] = year - df['rakennusvuosi']

In [156]:
#Lasketaan "myyntiaika"
df.columns

Index(['kohdenumero', 'tyyppi', 'huoneistoselitelmä', 'huoneita', 'koko',
       'kerrokset', 'rakennusvuosi', 'sauna', 'parveke', 'hissi', 'kunto',
       'taloyhtio', 'tontin_koko', 'tontin_omistus', 'hinta', 'url',
       'haettu_pvm', 'myyty_pvm', 'myyty', 'siirtyvät_liittymät',
       'tontin_tyyppi', 'tontin_nimi', 'tehokkuusluku', 'rakennukset',
       'vuokrasopimus_päättyy', 'sähköliittymä_siirtyy',
       'käyttö-_ja_luovutusrajoitukset', 'lisätietoja_tontista',
       'lisätietoja_vuokra-ajasta', 'rasitukset_ja_rasitteet', 'rasitteet',
       'y_tunnus', 'is_apartment', 'kaupunginosa', 'katuosoite', 'postinumero',
       'neliohinta', 'hoitovastike', 'oma_tontti', 'uudiskohde', 'kerros',
       'alin_kerros', 'ylin_kerros', 'ika'],
      dtype='object')

In [157]:
df['myyty_pvm'].value_counts()

myyty_pvm
24082024.00    358
17082024.00      5
12082024.00      5
5082024.00       1
2082024.00       1
Name: count, dtype: int64

In [158]:
df['haettu_pvm'] = pd.to_datetime(df['haettu_pvm'], format='%d%m%Y')
df['myyty_pvm'] = pd.to_datetime(df['myyty_pvm'], format='%d%m%Y')

In [159]:
mask_myyty_nan = df['myyty_pvm'].isna()

In [160]:
df['myyty_pvm_temp'] = df['myyty_pvm'].fillna(pd.Timestamp('1900-01-01'))
df['myyntiaika'] = df['myyty_pvm_temp'] -df['haettu_pvm']
df.loc[mask_myyty_nan, 'myyntiaika'] = pd.NaT
df.drop(columns =['myyty_pvm_temp'], inplace = True)

In [161]:
df['myyty_pvm'].value_counts()

myyty_pvm
2024-08-24    358
2024-08-17      5
2024-08-12      5
2024-08-05      1
2024-08-20      1
Name: count, dtype: int64

In [162]:
df['myyntiaika'] = df['myyty_pvm'] - df['haettu_pvm']

<h4>Neliovastike

In [163]:
df['hoitovastike'] = df['hoitovastike'].fillna(0)
df['koko'] = df['koko'].fillna(1)

In [164]:
df['neliovastike'] = df['hoitovastike'] / df['koko']

In [165]:
df = df.drop(columns = [
    'siirtyvät_liittymät','tontin_tyyppi','tontin_nimi', 'tehokkuusluku', 'rakennukset','sähköliittymä_siirtyy',
       'käyttö-_ja_luovutusrajoitukset', 'lisätietoja_tontista',
       'lisätietoja_vuokra-ajasta', 'rasitukset_ja_rasitteet', 'rasitteet' 
])

In [192]:
#Tehdään taloyhtiöstä oma df ja tallennetaan se CSV:ksi datascrapingia varten
ty = df.groupby(['y_tunnus','taloyhtio'], as_index = False).agg({'rakennusvuosi':'max'})
ty.to_csv("taloyhtiot.csv")

In [94]:
#Muutetaan huoneistojen y_tunnus-kenttä tilaan unknown, jos muoto on väärä
df.loc[(df['is_apartment'] == 1) &
(~df['y_tunnus'].str.contains(r'.\d+$',na = False)), 'y_tunnus'] = 'unknown'

In [95]:
#Muutetaan muiden kuin huoneistojen y-tunnus kenttä muotoon NaN
df.loc[df['is_apartment'] == 0, 'y_tunnus'] = np.nan

In [98]:
#Tallennetaan df analyysia varten
df.to_csv('oulu_apartment_listing_24082024.csv', index=False)